In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install neo4j
%pip install json-repair
%pip install py2neo
%pip install tree_sitter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install --upgrade langchain
%pip install llama-index --upgrade

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.13.post1
    Uninstalling llama-index-core-0.11.13.post1:
      Successfully uninstalled llama-index-core-0.11.13.post1
  Attempting uninstall: llama-index
    Found existing installation: llama-index 0.11.13
    Uninstalling llama-index-0.11.13:
      Successfully uninstalled llama-index-0.11.13
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip show llama-index
%pip install yfiles-jupyter-graphs
%pip install --upgrade yfiles-jupyter-graphs
%pip install --upgrade jupyterlab ipywidgets
%pip install jupyterlab
%pip install sqlfluff

Name: llama-index
Version: 0.11.14
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the ker

In [6]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import re

from dotenv import load_dotenv

load_dotenv()


/Users/larissakemicloud.com/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [17]:
os.environ["NEO4J_URI"] 
os.environ["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"]

# Agora você pode criar o gráfico sem passar os parâmetros diretamente
graph = Neo4jGraph()

In [7]:
# Função para limpar o código SQL
def clean_sql_code(sql_code):
    # Remover comentários de linha (--) e de bloco (/* ... */)
    sql_code = re.sub(r'(--[^\n]*)', '', sql_code)  # Remove comentários de linha
    sql_code = re.sub(r'(/\*[\s\S]*?\*/)', '', sql_code)  # Remove comentários de bloco
    
    # Remover espaços extras
    sql_code = re.sub(r'\s+', ' ', sql_code)  # Substitui múltiplos espaços por um único espaço
    sql_code = re.sub(r'\s*([,;()=<>])\s*', r'\1', sql_code)  # Remove espaços em torno de pontuação
    
    # Remover linhas vazias ou com espaços em branco
    sql_code = "\n".join([line for line in sql_code.splitlines() if line.strip()])
    
    return sql_code.strip()  # Remove espaços no início e no final

# Lista de caminhos de arquivos
file_paths = [
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoApiTokens.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupo.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupoUsuario.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoMenu.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoModuloTela.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoPermissao.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoPermissaoMenu.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoRecurso.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoTela.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoUsuario.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoUsuarioCliente.sql"
]

# Carregar os documentos
docs = []
for file_path in file_paths:
    loader = TextLoader(file_path, encoding="utf-8")
    docs.extend(loader.load())  # Adicionar documentos carregados à lista

# Limpar o código SQL antes de dividir
for doc in docs:
    doc.page_content = clean_sql_code(doc.page_content)

# Dividir os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

# Verificar os documentos divididos
print(f"Total de documentos divididos: {len(documents)}")

# Exibir os primeiros chunks para verificar a limpeza
for idx, chunk in enumerate(documents[:3]):  # Mostrar os primeiros 3 chunks
    print(f"Chunk {idx + 1}:\n{chunk.page_content}\n")

Total de documentos divididos: 20
Chunk 1:
﻿CREATE TABLE [dbo].[ControleDeAcessoApiTokens]([ID] INT IDENTITY(1,1)NOT NULL,[ID_Empresa] INT NOT NULL,[ID_Usuario] INT NOT NULL,[Token] VARCHAR(500)NOT NULL,[TokenExpiration] DATETIME NULL,[DataDaUltimaAlteracao] DATETIME NOT NULL,[UltimoUsuarioQueAlterou] INT NOT NULL,CONSTRAINT [PK_ControleDeAcessoApiTokens] PRIMARY KEY CLUSTERED([ID] ASC),CONSTRAINT [FK_ControleDeAcessoApiTokens_ControleDeAcessoUsuario] FOREIGN KEY([ID_Usuario])REFERENCES [dbo].[ControleDeAcessoUsuario]([ID]),CONSTRAINT

Chunk 2:
[FK_ControleDeAcessoApiTokens_Empresa] FOREIGN KEY([ID_Empresa])REFERENCES [dbo].[Empresa]([ID]),CONSTRAINT [CK_ControleDeAcessoApiTokens_Token] UNIQUE NONCLUSTERED([Token] ASC),CONSTRAINT [CK_ControleDeAcessoApiTokens_Usuario] UNIQUE NONCLUSTERED([ID_Empresa] ASC,[ID_Usuario] ASC));

Chunk 3:
﻿CREATE TABLE [dbo].[ControleDeAcessoGrupo]([ID] INT IDENTITY(1,1)NOT NULL,[ID_Empresa] INT NOT NULL,[Nome] NVARCHAR(250)NOT NULL,[Descricao] NVARCHAR(600

In [47]:
import re
import subprocess
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Função para limpar o código SQL (remover comentários, espaços extras etc.)
def clean_sql_code(sql_code):
    # Remover comentários de linha (--) e de bloco (/* ... */)
    sql_code = re.sub(r'(--[^\n]*)', '', sql_code)  # Remove comentários de linha
    sql_code = re.sub(r'(/\*[\s\S]*?\*/)', '', sql_code)  # Remove comentários de bloco
    
    # Remover espaços extras
    sql_code = re.sub(r'\s+', ' ', sql_code)  # Substitui múltiplos espaços por um único espaço
    sql_code = re.sub(r'\s*([,;()=<>])\s*', r'\1', sql_code)  # Remove espaços em torno de pontuação
    
    # Remover linhas vazias ou com espaços em branco
    sql_code = "\n".join([line for line in sql_code.splitlines() if line.strip()])
    
    return sql_code.strip()  # Remove espaços no início e no final

# Função para aplicar linting com sqlfluff
def lint_sql_code(sql_code):
    # Escreve o código SQL em um arquivo temporário para o sqlfluff processar
    with open("temp_sql_file.sql", "w") as file:
        file.write(sql_code)
    
    # Executa o sqlfluff para corrigir o código SQL
    result = subprocess.run(["sqlfluff", "fix", "temp_sql_file.sql"], capture_output=True, text=True)
    
    # Retorna o SQL corrigido (pós-linting)
    return result.stdout

# Lista de caminhos de arquivos
file_paths = [
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoApiTokens.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupo.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupoUsuario.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoMenu.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoModuloTela.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoPermissao.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoPermissaoMenu.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoRecurso.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoTela.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoUsuario.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoUsuarioCliente.sql"
]

# Carregar os documentos
docs = []
for file_path in file_paths:
    loader = TextLoader(file_path, encoding="utf-8")
    docs.extend(loader.load())  # Adicionar documentos carregados à lista

# Limpar e aplicar linting no código SQL antes de dividir
for doc in docs:
    original_content = doc.page_content
    # Passo 1: Limpeza básica do SQL
    cleaned_sql = clean_sql_code(original_content)
    
    # Passo 2: Aplicar linting (formatação com sqlfluff)
    linted_sql = lint_sql_code(cleaned_sql)
    
    # Atualizar o conteúdo do documento com o SQL limpo e corrigido
    doc.page_content = linted_sql
    
    # Verificar SQL original, limpo e lintado
    print(f"SQL Original:\n{original_content[:300]}...")  # Primeiros 300 caracteres
    print(f"SQL Limpo:\n{cleaned_sql[:300]}...")  # SQL após a limpeza
    print(f"SQL Lintado:\n{linted_sql[:300]}...\n")  # SQL após linting

# Dividir os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

# Verificar os documentos divididos
print(f"Total de documentos divididos: {len(documents)}")

# Exibir os primeiros chunks para verificar a limpeza e linting
for idx, chunk in enumerate(documents[:3]):  # Mostrar os primeiros 3 chunks
    print(f"Chunk {idx + 1}:\n{chunk.page_content}\n")


SQL Original:
﻿CREATE TABLE [dbo].[ControleDeAcessoApiTokens] (
    [ID]                      INT           IDENTITY (1, 1) NOT NULL,
    [ID_Empresa]              INT           NOT NULL,
    [ID_Usuario]              INT           NOT NULL,
    [Token]                   VARCHAR (500) NOT NULL,
    [TokenExpirati...
SQL Limpo:
﻿CREATE TABLE [dbo].[ControleDeAcessoApiTokens]([ID] INT IDENTITY(1,1)NOT NULL,[ID_Empresa] INT NOT NULL,[ID_Usuario] INT NOT NULL,[Token] VARCHAR(500)NOT NULL,[TokenExpiration] DATETIME NULL,[DataDaUltimaAlteracao] DATETIME NOT NULL,[UltimoUsuarioQueAlterou] INT NOT NULL,CONSTRAINT [PK_ControleDeAc...
SQL Lintado:
==== finding fixable violations ====
...

SQL Original:
﻿CREATE TABLE [dbo].[ControleDeAcessoGrupo] (
    [ID]                      INT            IDENTITY (1, 1) NOT NULL,
    [ID_Empresa]              INT            NOT NULL,
    [Nome]                    NVARCHAR (250) NOT NULL,
    [Descricao]               NVARCHAR (600) NULL,
    [Tipo]          

In [48]:
documents[0]

Document(metadata={'source': '/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoApiTokens.sql'}, page_content='==== finding fixable violations ====')

In [ ]:
# Obter as chaves da API
openai_api_key = os.getenv("OPENAI_API_KEY")
llama_api_key = os.getenv("LLAMA_API_KEY")

# Definir qual LLM será utilizado
llm_type = os.getenv("LLM_TYPE", "openai")  # Valor padrão "openai" se não estiver definido

if llm_type == "ollama":
    # Usar o modelo LLaMA/Ollama e passar a chave de API da LLaMA
    llm = ChatOllama(model="llama3.1", temperature=0, api_key=llama_api_key)
else:
    # Usar o modelo OpenAI e passar a chave de API da OpenAI
    llm = ChatOpenAI(temperature=0, model="gpt-4o", api_key=openai_api_key)

# Usar o modelo LLM no transformador de grafos
llm_transformer = LLMGraphTransformer(llm=llm)

# Converter documentos para o formato de grafo
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [14]:
from langchain.schema import Document

# Obter as chaves da API
openai_api_key = os.getenv("OPENAI_API_KEY")
llama_api_key = os.getenv("LLAMA_API_KEY")

# Definir qual LLM será utilizado
llm_type = os.getenv("LLM_TYPE", "openai")  # Valor padrão "openai" se não estiver definido

if llm_type == "ollama":
    # Usar o modelo LLaMA/Ollama e passar a chave de API da LLaMA
    llm = ChatOllama(model="llama3.1", temperature=0, api_key=llama_api_key)
else:
    # Usar o modelo OpenAI e passar a chave de API da OpenAI
    llm = ChatOpenAI(temperature=0, model="gpt-4o", api_key=openai_api_key)

# Criar o prompt especializado para SQL
def generate_sql_prompt(sql_code):
    return f"""
    Você está analisando um código SQL. Sua tarefa é identificar as seguintes informações:
    1. Tabelas (com nomes).
    2. Colunas em cada tabela.
    3. Chaves primárias e chaves estrangeiras.
    4. Relacionamentos entre tabelas.

    Aqui está o código SQL:
    {sql_code}

    Extraia essas informações de forma clara e concisa antes de transformá-las em um grafo.
    """

# Função para processar e transformar cada documento SQL em grafo
def process_sql_documents(documents, llm, llm_transformer):
    graph_documents = []

    for doc in documents:
        # Gerar o prompt especializado para SQL
        prompt = generate_sql_prompt(doc.page_content)
        
        # Montar a mensagem no formato correto para o LLM
        messages = [
            {"role": "system", "content": "Você é um assistente especializado em SQL."},
            {"role": "user", "content": prompt}
        ]
        
        # Usar o LLM diretamente para interpretar o SQL e extrair as informações
        extracted_info = llm(messages=messages)  # Passando a lista de mensagens
        
        # Acessar diretamente o conteúdo da resposta
        extracted_content = extracted_info.content
        
        # Criar um objeto Document a partir do conteúdo extraído
        document = Document(page_content=extracted_content)
        
        # Usar o LLMGraphTransformer para transformar em grafo
        graph_doc = llm_transformer.convert_to_graph_documents([document])
        graph_documents.extend(graph_doc)
    
    return graph_documents

# Usar o LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

# Converter documentos SQL para o formato de grafo, extraindo as informações importantes primeiro
graph_documents = process_sql_documents(documents, llm, llm_transformer)

# Verificar os primeiros documentos de grafo gerados
for idx, graph_doc in enumerate(graph_documents[:3]):
    print(f"Grafo {idx + 1}: {graph_doc}")

Grafo 1: nodes=[Node(id='Controledeacessoapitokens', type='Table', properties={}), Node(id='Controledeacessousuario', type='Table', properties={})] relationships=[Relationship(source=Node(id='Controledeacessoapitokens', type='Table', properties={}), target=Node(id='Controledeacessousuario', type='Table', properties={}), type='FOREIGN_KEY', properties={})] source=Document(metadata={}, page_content='Claro! Vamos analisar o código SQL fornecido e extrair as informações solicitadas.\n\n### 1. Tabelas\n- **ControleDeAcessoApiTokens**\n\n### 2. Colunas em cada tabela\n- **ControleDeAcessoApiTokens**\n  - ID (INT, IDENTITY(1,1), NOT NULL)\n  - ID_Empresa (INT, NOT NULL)\n  - ID_Usuario (INT, NOT NULL)\n  - Token (VARCHAR(500), NOT NULL)\n  - TokenExpiration (DATETIME, NULL)\n  - DataDaUltimaAlteracao (DATETIME, NOT NULL)\n  - UltimoUsuarioQueAlterou (INT, NOT NULL)\n\n### 3. Chaves primárias e chaves estrangeiras\n- **Chave Primária**\n  - **ControleDeAcessoApiTokens**\n    - PK_ControleDeAce

In [15]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Controledeacessoapitokens', type='Table', properties={}), Node(id='Controledeacessousuario', type='Table', properties={})], relationships=[Relationship(source=Node(id='Controledeacessoapitokens', type='Table', properties={}), target=Node(id='Controledeacessousuario', type='Table', properties={}), type='FOREIGN_KEY', properties={})], source=Document(metadata={}, page_content='Claro! Vamos analisar o código SQL fornecido e extrair as informações solicitadas.\n\n### 1. Tabelas\n- **ControleDeAcessoApiTokens**\n\n### 2. Colunas em cada tabela\n- **ControleDeAcessoApiTokens**\n  - ID (INT, IDENTITY(1,1), NOT NULL)\n  - ID_Empresa (INT, NOT NULL)\n  - ID_Usuario (INT, NOT NULL)\n  - Token (VARCHAR(500), NOT NULL)\n  - TokenExpiration (DATETIME, NULL)\n  - DataDaUltimaAlteracao (DATETIME, NOT NULL)\n  - UltimoUsuarioQueAlterou (INT, NOT NULL)\n\n### 3. Chaves primárias e chaves estrangeiras\n- **Chave Primária**\n  - **ControleDeAcessoApiTokens**\n    - PK_Contro

In [16]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [37]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (s)-[r:MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [38]:
graph.query("SHOW INDEXES")

TypeError: 'str' object is not callable

In [27]:
# Cria uma vectorstore
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [39]:
schema_data = print(graph.schema)

Node properties:
Document {id: STRING, text: STRING, embedding: LIST}
Column {id: STRING}
Table {id: STRING}
Primary_key {id: STRING}
Foreign_key {id: STRING}
Relationship properties:

The relationships:
(:Document)-[:MENTIONS]->(:Table)
(:Document)-[:MENTIONS]->(:Column)
(:Document)-[:MENTIONS]->(:Foreign_key)
(:Document)-[:MENTIONS]->(:Primary_key)
(:Column)-[:PRIMARY_KEY]->(:Table)
(:Column)-[:REFERENCES]->(:Table)
(:Column)-[:REFERENCES]->(:Column)
(:Column)-[:FOREIGN_KEY]->(:Column)
(:Table)-[:HAS_COLUMN]->(:Column)
(:Table)-[:FOREIGN_KEY]->(:Table)
(:Table)-[:FOREIGN_KEY]->(:Column)
(:Table)-[:REFERENCES]->(:Table)
(:Table)-[:PRIMARY_KEY]->(:Column)
(:Table)-[:HAS_PRIMARY_KEY]->(:Primary_key)
(:Table)-[:HAS_FOREIGN_KEY]->(:Foreign_key)
(:Foreign_key)-[:REFERENCES]->(:Table)


In [29]:
# Perguntas básicas sobre o Graph
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Definir o template para geração de Cypher
CYPHER_GENERATION_TEMPLATE = """Task: Gerar uma instrução Cypher para consultar um banco de dados gráfico com base no esquema SQL fornecido. 
Instruções:
- Verifique o esquema do banco de dados SQL consultando o Neo4j para obter os nomes das tabelas e colunas.
- Construa uma consulta SQL com base nas informações extraídas do gráfico.
- Certifique-se de verificar os relacionamentos (chaves estrangeiras) no banco de dados gráfico para garantir que a query SQL seja válida.
- Não inclua explicações ou desculpas em suas respostas. Apenas forneça a query SQL gerada.
- Não responda a perguntas que solicitem qualquer coisa além da construção de uma instrução Cypher.
- Não inclua nenhum texto além da instrução Cypher gerada.
Task: Gerar uma instrução SQL válida com base nas informações de um banco de dados gráfico Neo4j. 
Schema:
{schema}

The question is:
{query}"""

# Criar o template para o prompt
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "query"], template=CYPHER_GENERATION_TEMPLATE
)

# Conectar ao modelo da OpenAI
llm = ChatOpenAI(temperature=0)

# Conectar ao Neo4j e criar o chain, permitindo solicitações "perigosas"
chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,  # Passa a conexão ao banco de dados Neo4j
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests=True,  # Permitir consultas perigosas
)

# Obter o esquema do banco de dados Neo4j automaticamente
schema_data = graph.schema  # Supondo que graph.schema retorne o esquema em texto

# Definir a query
query = "Quantas relacoes no total a tabela Controledeacessomenu possui?"

# Executar a cadeia para gerar a consulta Cypher a partir do schema e da query
result = chain.run({"schema": schema_data, "query": query})

# Imprimir a consulta Cypher gerada
print(f"Cypher gerada: {result}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Table {id: "Controledeacessomenu"})-[:HAS_COLUMN]->(c:Column)
RETURN COUNT(c) as total_relacoes;
Full Context:
[{'total_relacoes': 11}]

> Finished chain.
Cypher gerada: A tabela Controledeacessomenu possui um total de 11 relações.


In [56]:
import os
import re
from typing import List, Dict
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

# Configurações via variáveis de ambiente
neo4j_uri = os.getenv("NEO4J_URI")  # URI do Neo4j
neo4j_username = os.getenv("NEO4J_USERNAME")  # Nome de usuário do Neo4j
neo4j_password = os.getenv("NEO4J_PASSWORD")  # Senha do Neo4j

# Conectar ao Neo4j usando as variáveis de ambiente
graph = Neo4jGraph(
    url=neo4j_uri,
    username=neo4j_username,
    password=neo4j_password
)

# Definir o template para geração de Cypher
CYPHER_GENERATION_TEMPLATE = """Task: Gerar uma instrução Cypher para consultar um banco de dados gráfico com base no esquema SQL fornecido. 
Instruções:
- Verifique o esquema do banco de dados SQL consultando o Neo4j para obter os nomes das tabelas e colunas.
- Construa uma consulta SQL com base nas informações extraídas do gráfico.
- Certifique-se de verificar os relacionamentos (chaves estrangeiras) no banco de dados gráfico para garantir que a query SQL seja válida.
- Não inclua explicações ou desculpas em suas respostas. Apenas forneça a query SQL gerada.
- Não responda a perguntas que solicitem qualquer coisa além da construção de uma instrução Cypher.
- Não inclua nenhum texto além da instrução Cypher gerada.
Task: Verificar se a tabela '{table}' e as colunas '{columns}' existem no banco de dados gráfico Neo4j. 
Schema:
{schema}

The question is:
{query}"""

# Criar o template para o prompt
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "table", "columns", "query"], template=CYPHER_GENERATION_TEMPLATE
)

# Conectar ao modelo da OpenAI
llm = ChatOpenAI(temperature=0)

# Conectar ao Neo4j e criar o chain, permitindo solicitações "perigosas"
chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,  # Conexão com o banco de dados Neo4j
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests=True  # Permitir consultas perigosas
)

# Função para gerar a query SQL inicial com base na pergunta
def generate_initial_sql_query(question: str) -> str:
    SQL_GENERATION_TEMPLATE = """Task: Gerar uma instrução SQL válida com base na pergunta do usuário.
    Instruções:
    - Gere a query SQL com base na pergunta fornecida, assumindo que todas as tabelas e colunas estão disponíveis.
    - A query não deve incluir SELECT *, deve especificar todas as colunas.
    - A questão é:
    {query}
    """
    messages = [
        SystemMessage(content=SQL_GENERATION_TEMPLATE.format(query=question))
    ]
    response = llm.invoke(messages)
    return response.content

# Obter o esquema do banco de dados Neo4j automaticamente
schema_data = graph.schema  # Supondo que graph.schema retorne o esquema em texto

# Função para verificar o esquema no Neo4j com Cypher e retornar confirmação
def verify_schema_with_chain(schema: str, table: str, columns: List[str]) -> str:
    # Executa o chain de verificação de Cypher
    column_list = ", ".join(columns)
    result = chain.run({"schema": schema, "table": table, "columns": column_list, "query": f"Verifique se a tabela '{table}' e as colunas {column_list} existem."})
    return result

# Função para extrair tabelas e colunas da query SQL gerada
def extract_tables_and_columns(sql_query: str) -> Dict[str, List[str]]:
    # Aqui você pode ajustar a lógica de regex para extrair as tabelas e colunas da query SQL
    table_column_mapping = {}
    table_matches = re.findall(r'FROM\s+(\w+)', sql_query, re.IGNORECASE)
    column_matches = re.findall(r'SELECT\s+(.*?)\s+FROM', sql_query, re.IGNORECASE)

    for table, columns in zip(table_matches, column_matches):
        columns_list = [col.strip() for col in columns.split(",")]
        table_column_mapping[table] = columns_list

    return table_column_mapping

# Função principal para gerar a query SQL final
def generate_final_sql(question: str):
    # 1. Gerar a query SQL inicial
    sql_query = generate_initial_sql_query(question)
    print(f"Query SQL inicial gerada:\n{sql_query}")
    
    # 2. Extrair tabelas e colunas da query SQL gerada
    tables_and_columns = extract_tables_and_columns(sql_query)
    
    # 3. Verificar cada tabela e suas colunas no Neo4j
    for table, columns in tables_and_columns.items():
        result = verify_schema_with_chain(schema_data, table, columns)
        if "não existem" in result:
            raise ValueError(f"A tabela '{table}' ou as colunas '{columns}' não existem no banco de dados Neo4j.")
    
    # 4. Se tudo estiver correto, retorne a query SQL final
    return sql_query

# Exemplo de uso
try:
    question = "Qual é a quantidade de pedidos no ano?"
    final_sql = generate_final_sql(question)
    print(f"Query SQL final gerada:\n{final_sql}")
except ValueError as e:
    print(f"Erro: {e}")

2024-09-29 20:12:25,654 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Query SQL inicial gerada:
```sql
SELECT COUNT(id_pedido) AS quantidade_pedidos
FROM pedidos
WHERE YEAR(data_pedido) = YEAR(CURRENT_DATE)
```


> Entering new GraphCypherQAChain chain...


2024-09-29 20:12:26,493 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE t.id = 'pedidos' AND c.id = 'id_pedido'
RETURN COUNT(c) AS quantidade_pedidos;
Full Context:
[{'quantidade_pedidos': 0}]


2024-09-29 20:12:27,562 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.
Query SQL final gerada:
```sql
SELECT COUNT(id_pedido) AS quantidade_pedidos
FROM pedidos
WHERE YEAR(data_pedido) = YEAR(CURRENT_DATE)
```
